# Compute rank order statistics on human annotated importance scores

In [ ]:
cd ../

In [2]:
from scipy.stats import kendalltau, spearmanr
from scipy.stats import rankdata
import numpy as np

In [3]:
import sys
sys.path.append('src/')
from tools.io import load_tvsum_mat

In [4]:
data = load_tvsum_mat('./data/raw/tvsum/ydata-tvsum50.mat')

In [5]:
def get_rc_func(metric):
    if metric == 'kendalltau':
        f = lambda x, y: kendalltau(rankdata(-x), rankdata(-y))
    elif metric == 'spearmanr':
        f = lambda x, y: spearmanr(x, y)
    else:
        raise RuntimeError
    return f

class RankCorrelationEvaluator(object):
    
    def __call__(self):
        res = []
        for d in data:
            user_anno = d['user_anno'].T
            N = user_anno.shape[1]

            D = []
            mean_tau = []
            min_tau = []
            max_tau = []

            pred_x = self.get_score(d['video'])
            D = [self.rc_func(x, pred_x)[0] for x in user_anno]

            res.append({'video': d['video'],
                        'mean': np.mean(D),
                       'min': np.min(D), 
                       'max': np.max(D), 
                        'cc': np.asarray(D)
                       })
        return res

class HumanEvaluator(RankCorrelationEvaluator):
    def __init__(self, metric):
        self.rc_func = get_rc_func(metric)
    
    def __call__(self):
        res = []
        for d in data:
            user_anno = d['user_anno'].T
            N = user_anno.shape[1]
            
            max_rc = []
            min_rc = []
            avr_rc = []
            rc = []
            for i, x in enumerate(user_anno):
                R = [self.rc_func(x, user_anno[j])[0] for j in range(len(user_anno)) if j != i]
                
                max_rc.append(max(R))
                min_rc.append(min(R))
                avr_rc.append(np.mean(R))
                rc += R
                
            res.append({'video': d['video'],
                        'mean': np.mean(avr_rc),
                       'min': np.mean(min_rc), 
                       'max': np.mean(max_rc), 
                        'cc': np.asarray(rc)
                       })
        return res
    
class RandomEvaluator(RankCorrelationEvaluator):
    def __init__(self, metric):
        self.rc_func = get_rc_func(metric)
        
        rand_scores = {}
        for d in data:
            user_anno = d['user_anno'].T
            N = user_anno.shape[1]
            rand_scores[d['video']] = np.random.random((N,))
            
        self.rand_scores = rand_scores
            
    def get_score(self, v_id):
        return self.rand_scores[v_id]


In [6]:
metric = 'spearmanr'
human_res = HumanEvaluator(metric)()
mean_arr = np.asarray([x['mean'] for x in human_res])
print('human'+': mean %.3f'%(np.mean(mean_arr)))

human: mean 0.204


In [7]:
metric = 'kendalltau'
human_res = HumanEvaluator(metric)()
mean_arr = np.asarray([x['mean'] for x in human_res])
print('human'+': mean %.3f'%(np.mean(mean_arr)))

human: mean 0.177
